In [1]:
from win32com import client as wc
from pathlib import Path
from contextlib import contextmanager
import json

In [6]:
with open("action_parameterset.json", "r", encoding='utf-8') as f:
    action_keys = json.load(f)

In [2]:
def get_absolute_path(path):
    name = Path(path)
    return name.absolute().as_posix()

In [3]:
class HwpApi:
    def __init__(self, api=None):
        if not api:
            
            api = wc.gencache.EnsureDispatch("HWPFrame.HwpObject")
        self.api = api
    
    def set_visible(self, is_visible=True, window_i=0):
        self.api.XHwpWindows.Item(window_i).Visible = is_visible
    
    
    def get_filepath(self):
        doc = self.api.XHwpDocuments.Active_XHwpDocument
        return doc.FullName
    
    def open(self, path:str):
        name = get_absolute_path(path)
        self.api.Open(name)
        return name
    
    def save(self, path=None, save_format="HWP"):
        if not path:
            self.api.Save()
            return self.get_filepath()
        name = get_absolute_path(path)
        self.api.SaveAs(name, save_format)
        return name
    
    def close(self):
        self.api.Run("Close")
        
    def create_action(self, action_key:str):
        """"""
        return Action(self, action_key)
    
    def create_parameterset(self, action_key:str):
        """it is not working and I do not know why. I guess __getattr__method is not work as I expect."""
        return ParameterSet(self, action_key)

In [4]:
# we can wrap object and autocompliation and I think it is a problem.

class ParameterSet(object):
    
    def __init__(self, hwpapi, action_key):
        pset_key = action_keys.get(action_key, None)
        
        self._pset_key = pset_key
        self.api = hwpapi.api
        
        pset = getattr(self.api.HParameterSet, f"H{pset_key}")
        self.api.HAction.GetDefault(action_key, pset.HSet)
        self.methods = []
        self._pset = pset
        
    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        return getattr(self._pset, attr)
    
    def __dir__(self):
        return self.methods + dir(self._pset)
    

In [5]:
class Action:
    
    def __init__(self, hwpapi:HwpApi, action_key:str):
        
        self.action_key = action_key
        self.hwpapi = hwpapi
        
        # create action and set
        self.act = hwpapi.api.CreateAction(action_key)
        self._pset = self.act.CreateSet()
        self.act.GetDefault(self._pset)
              
    
    def apply_pset(self, pset:ParameterSet):
        
        self._pset.Merge(pset.HSet)
        return self
    
    def set_parameter(self, key:str, value):
        
        self._pset.SetItem(key, value)
        return self
    
    def get_parameter(self, key:str):
        
        return self._pset.Item(key)
    
    def run(self):
        self.act.Execute(self._pset)
        return self
    
    def create_pset(self):
        pset_key = action_keys.get(self.action_key, None)
        self._pset_key = pset_key
        pset = getattr(self.hwpapi.api.HParameterSet, f"H{pset_key}")
        self.hwpapi.api.HAction.GetDefault(self.action_key, pset.HSet)
        return pset
        
    @contextmanager
    def pset(self):
        parameterset = self.create_pset()
        yield parameterset
        self.apply_pset(parameterset)
    


In [7]:
hwpapi = HwpApi()
hwpapi.set_visible()

In [44]:
act = hwpapi.create_action("InsertText")

In [9]:
pset = act.create_pset()

In [10]:
pset2 = hwpapi.api.HParameterSet.HInsertText

In [11]:
pset3 = hwpapi.create_parameterset("InsertText")

In [12]:
pset.Text = "test1"

In [13]:
pset2.Text = "test2"

In [14]:
pset3.Text = "test3"

In [15]:
for p in [pset, pset2, pset3]:
    act.apply_pset(p).run()

In [46]:
with act.pset() as p:
    p.Text="TextTest"
act.run()

## ParameterSet Key를 Action Table에서 얻어오기

In [17]:
import pandas as pd

In [18]:
tables = pd.read_html(r"C:\Users\freed\Documents\python_projects\HwpApi\action_table\Action Table_modified.htm")

In [19]:
action_table = pd.DataFrame(tables[1].loc[1:].values, columns=tables[1].loc[0]).query("'Action ID' != `Action ID`")
action_table["ParameterSet"] = action_table["ParameterSet ID"].str.replace("[\+\*\-]", "")
action_table

C:\Users\freed\AppData\Local\Temp\ipykernel_316\2958487354.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  action_table["ParameterSet"] = action_table["ParameterSet ID"].str.replace("[\+\*\-]", "")


,Action ID,ParameterSet ID,Description,비고,ParameterSet
0,AddHanjaWord,+,한자단어 등록,NaN,
1,AllReplace,FindReplace*,모두 바꾸기,NaN,FindReplace
2,AQcommandMerge,UserQCommandFile*,입력 자동 명령 파일 저장/로드 \t(한글메뉴의 [도구-빠른 교정-빠른 교정내용]에...,한글2007에 새로 추가,UserQCommandFile
3,AutoChangeHangul,-,낱자모 우선,NaN,
4,AutoChangeRun,-,동작,NaN,
...,...,...,...,...,...
748,VoiceCommand,+,음성 명령 동작,NaN,
749,VoiceCommand Config,-,음성 명령 설정,NaN,
750,VoiceCommand Resume,-,음성 명령 레코딩 시작,NaN,
751,VoiceCommand Stop,-,음성 명령 레코딩 중지,NaN,


In [20]:
pset_keys = action_table[["Action ID", "ParameterSet"]].set_index("Action ID")["ParameterSet"].to_dict()

In [21]:
with open("action_parameterset.json", "w", encoding='utf-8') as f:
    json.dump(pset_keys, f)